## Submission 8
I realized I never removed outliers because I was trying to keep as much data as possible

(Submission 7 notebook copied, which includes train3 data and test2 - this has been my best submission so far)
I will do a box plot to see if there are outliers of Saleprice to remove and then run my linear regression again to see if that improves the prediction

I will comment on anything new

Since this was my last notebook/ submission, I also used this notebook to make many of my visuals for my presentation

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import metrics
import warnings; warnings.simplefilter('ignore')

In [2]:
train3 = pd.read_csv('./datasets/train3.csv')
test2 = pd.read_csv('./datasets/test2.csv')
train3.set_index('Id', inplace=True)
test2.set_index('Id', inplace=True)

FileNotFoundError: [Errno 2] File ./datasets/train3.csv does not exist: './datasets/train3.csv'

In [ ]:
train3.shape

In [ ]:
plt.figure(figsize=(10,10))
plt.boxplot(train3['SalePrice'])
plt.title('Distribution of Sale Price', size=20)
plt.ylabel('Price', size=15);
plt.savefig('./images/price_dist_outliers.png')
#There seems to be a lot of outliers in the higher sale prices
#https://towardsdatascience.com/ways-to-detect-and-remove-the-outliers-404d16608dba
#https://chartio.com/resources/tutorials/how-to-save-a-plot-to-a-file-using-matplotlib/

In [ ]:
#Determine Z score for Sale Price in the train dataset
from scipy import stats
z = np.abs(stats.zscore(train3['SalePrice']))
print(z)

In [ ]:
#Outliers are anything more than 3 zscores/ 3 standard deviations away from the mean
threshold = 3
outliers = np.where(z>3)
outliers

In [ ]:
train3.shape

In [ ]:
#Create a column for outliers, 1 is an outlier, 0 is not
train3['outliers'] = np.where(z>3, 1,0)
train3.head()

In [ ]:
#Keep only the rows which have a 0 value for outliers column, meaning they are not an outlier
train3 = train3[train3['outliers']==0]
train3.shape
#This dropped 33 outliers!

In [ ]:
plt.figure(figsize=(10,10))
plt.boxplot(train3['SalePrice'])
plt.title('Distribution of Sale Price', size=20)
plt.ylabel('Price', size=15)
plt.savefig('./images/price_dist_no_outliers.png');

In [ ]:
#I also noticed there were two extra points that are very off
#When I originally made the Gr Liv Area v. Sale Price scatter plot, the two extra points had Gr Liv Areas over 5000 square feet
#I dropped those two points as well
train3 = train3[train3['Gr Liv Area'] < 4900]
train3.shape

In [ ]:
train_nb_dum = pd.get_dummies(train3['Neighborhood'])
train_nb_dum['SalePrice'] = train3['SalePrice']
plt.figure(figsize=(10,10))
sns.heatmap(train_nb_dum.corr()[['SalePrice']], annot=True, cmap='YlGnBu');

In [ ]:
#Mean sale price by neighborhood
train3['SalePrice'].groupby(train3['Neighborhood']).mean().sort_values(ascending=False)

In [ ]:
#Create a bar graph of the mean sale price per neighborhood
x_plot = sorted([hood for hood in train3['Neighborhood'].unique()])
y_plot = train3['SalePrice'].groupby(train3['Neighborhood']).mean()
plt.figure(figsize=(10,10))
sns.barplot(x_plot, y_plot)
plt.xticks(rotation=45)
plt.title('Average Sale Price per Neighborhood', size=20)
plt.xlabel('Neighborhood', size=15)
plt.ylabel('Average Sale Price ($)', size=15)
plt.savefig('./images/avg_price_neighborhood.png');

In [ ]:
#Pick NoRidge, NridgHt, StoneBr - a lot of the outliers were in these neighborhoods, the correlations are now lower
#Somerst now has a higher correlation than StoneBr so I will add that one to the neighborhood dummy column
train3['4nbs'] = train_nb_dum['NoRidge']+train_nb_dum['NridgHt']+train_nb_dum['StoneBr']+train_nb_dum['Somerst']
test_nb_dum = pd.get_dummies(test2['Neighborhood'])
test2['4nbs'] = test_nb_dum['NoRidge']+test_nb_dum['NridgHt']+test_nb_dum['StoneBr']+test_nb_dum['Somerst']

In [ ]:
test2['4nbs'].isnull().sum()

In [ ]:
def r2_adj(x, y, k):
    lr = LinearRegression()
    lr.fit(x, y)
    null_pred=y.mean()
    null_resids = y-null_pred
    null_sse = (null_resids**2).sum()
    pred = lr.predict(x)
    resids = y - pred
    sse=(resids**2).sum()
    r2 = 1-((sse) / (null_sse))
    n = len(y)
    r2_adj = 1 - (1-r2)*(n-1)/(n-k-1)
    return r2_adj

def run_metrics(x,y, k):
    lr = LinearRegression()
    lr.fit(x, y)
    pred = lr.predict(x)
    resids = y - pred
    r2 = lr.score(x,y)
    mae = metrics.mean_absolute_error(y, pred)
    sse = (resids**2).sum()
    rmse = metrics.mean_squared_error(y, pred, squared=False)
    mse = metrics.mean_squared_error(y, pred)
    r2a = r2_adj(x,y,k)
    print(f'Mean Adjusted errors: {mae}')
    print(f'Sum Squared Errors: {sse}')
    print(f'Mean Square Errors: {mse}' )
    print(f'Root Mean Square Errors: {rmse}')
    print(f'R2: {r2}')
    print(f'Adjusted R2: {r2a}')

In [ ]:
features = ['Overall Qual', 'Gr Liv Area', 'Garage Area', 'Total Bsmt SF', '1st Flr SF', 'Year Built', 'Full Bath', 
            'Qual_Cond', 'lot_liv', 'TotRms AbvGrd', 'Garage Yr Blt', 'Overall Cond', 'Kitchen AbvGr', 'Bedroom AbvGr', 
           'high_qual_fp', 'paved_street', '4nbs']
X = train3[features]
y = train3['SalePrice']
run_metrics(X,y,17)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)
linreg=LinearRegression()
linreg.fit(X_train, y_train)

In [ ]:
print(f'R2 Train {linreg.score(X_train, y_train)}')
print(f'RMSE Train {metrics.mean_squared_error(y_train, linreg.predict(X_train), squared=False)}')
print(f'R2 Test {linreg.score(X_test, y_test)}')
print(f'RMSE Test {metrics.mean_squared_error(y_test, linreg.predict(X_test), squared=False)}')

In [ ]:
#lasso coding commented in submission7 journal
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso, LassoCV

In [ ]:
sc = StandardScaler()
Z_train = sc.fit_transform(X_train)
Z_test = sc.transform(X_test)

In [ ]:
linreg.fit(Z_train, y_train)
linreg.score(Z_train, y_train)

In [ ]:
linreg.score(Z_test, y_test)

In [ ]:
l_alphas = np.logspace(0,5, 100)
lasso_cv = LassoCV(alphas = l_alphas, cv=5)
lasso_cv.fit(Z_train, y_train)
print(f'Accuracy Lasso train: {lasso_cv.score(Z_train, y_train)}')
print(f'Accuracy Lasso test: {lasso_cv.score(Z_test, y_test)}')

In [ ]:
lasso_cv.alpha_

In [ ]:
lasso_cv.coef_

In [ ]:
test2_sc = sc.fit_transform(test2[features])
test2['SalePrice2'] = lasso_cv.predict(test2_sc)
test2.head()

In [ ]:
#linreg output without outliers
output8 = test2[['SalePrice']]
output8.rename(columns = {'SalePrice2':'SalePrice'}, inplace = True)
output8.head()

In [ ]:
output8.to_csv('./datasets/dana_submission_8.csv')

In [ ]:
#LINEAR coefficient dataframe
train3_coefs = train3[features]
train3_coefs.head()
coefs = pd.DataFrame({'column': train3_coefs.columns, 'coef': linreg.coef_})
coefs

In [ ]:
plt.scatter(y_train, lasso_cv.predict(Z_train));

In [ ]:
plt.scatter(y_train, lasso_cv.predict(Z_train));

In [ ]:
#LASSO coeffieicents
pd.DataFrame({'column':X.columns, 'coef':lasso_cv.coef_})

In [ ]:
#We can remove all features which have a coefficient of 0
#This means the LASSO regression said they were not important
features = ['Overall Qual', 'Gr Liv Area', 'Garage Area', 'Total Bsmt SF', '1st Flr SF', 'Year Built', 
            'Qual_Cond', 'Garage Yr Blt', 'Kitchen AbvGr', 'high_qual_fp', '4nbs']
X = train3[features]
y = train3['SalePrice']
run_metrics(X,y,11)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)
linreg=LinearRegression()
linreg.fit(X_train, y_train)

In [ ]:
print(f'R2 train {linreg.score(X_train, y_train)}')
print(f'R2 test{linreg.score(X_test, y_test)}')
print()
print(f'RMSE train {metrics.mean_squared_error(y_train, linreg.predict(X_train), squared=False)}')
print(f'RMSE test {metrics.mean_squared_error(y_test, linreg.predict(X_test), squared=False)}')

In [ ]:
#Actual price vs Predicted price
#The diagonal line is where x=y, this is where a point would be if the model predicted the actual sale price
plt.figure(figsize=(10,10))
plt.scatter(y_train, linreg.predict(X_train), alpha=0.5)
plt.title('Actual vs. Predicted Price for Training Data', size = 20)
plt.xlabel('Actual Price', size=15)
plt.xticks(rotation=30)
plt.ylabel('Predicted Price', size=15)
lineStart = y_train.min() 
lineEnd = y_train.max()  

plt.plot([lineStart, lineEnd], [lineStart, lineEnd], 'k-', color = 'r')
plt.xlim(lineStart, lineEnd)
plt.ylim(lineStart, lineEnd)
plt.show()
plt.savefig('./images/actual_pred_train.png');
#Noah sent me this link! https://stackoverflow.com/questions/40516661/adding-line-to-scatter-plot-using-pythons-matplotlib

In [ ]:
plt.figure(figsize=(10,10))
sns.regplot(y_train,
            linreg.predict(X_train),
            scatter_kws={'alpha' : 0.5}, 
            line_kws={'color':'red'}).set_title('Actual vs. Predicted Price for Training Data', size = 20)
plt.ylabel('Predicted Price', size=15)
plt.xlabel('Actual Price', size=15);
#Thank you Noah!!!!
#I want a line where y=x to show how close the predictions are to the actual value - this is the right plot for that, see above
#Good to know this type of plot exists!

In [ ]:
plt.figure(figsize=(10,10))
sns.regplot(y_test,
            linreg.predict(X_test),
            scatter_kws={'alpha' : 0.5, 'color':'red'}, 
            line_kws={'color':'blue'}).set_title('Actual vs. Predicted Price for Testing Data', size = 20)
plt.ylabel('Predicted Price', size=15)
plt.xlabel('Actual Price', size=15);
#Thank you Noah!!!!
#Same as the other SNS plot, this is not the right one for what I was trying to show

In [ ]:
#Actual price vs Predicted price
#The diagonal line is where x=y, this is where a point would be if the model predicted the actual sale price
plt.figure(figsize=(10,10))
plt.scatter(y_test, linreg.predict(X_test), alpha=0.5, color='red')
plt.title('Actual vs. Predicted Price for Testing Data', size = 20)
plt.xlabel('Actual Price', size=15)
plt.xticks(rotation=30)
plt.ylabel('Predicted Price', size=15)
lineStart = y_test.min() 
lineEnd = y_test.max()  

plt.plot([lineStart, lineEnd], [lineStart, lineEnd], 'k-', color = 'b')
plt.xlim(lineStart, lineEnd)
plt.ylim(lineStart, lineEnd)
plt.show()
plt.savefig('./images/actual_pred_test.png');
#Noah sent me this link! https://stackoverflow.com/questions/40516661/adding-line-to-scatter-plot-using-pythons-matplotlib

In [ ]:
#Scatter plot of Above Grade Living vs Sale Price
plt.figure(figsize=(10,10))
plt.scatter(train3['Gr Liv Area'], train3['SalePrice'])
plt.title('Above Grade Living Area vs Sale Price', size = 20)
plt.xlabel('Above Grade Living Area (Sq Ft)', size=15)
plt.ylabel('Sale Price ($)', size=15);

In [ ]:
#Scatter plot of Year Built vs Sale Price
plt.figure(figsize=(10,10))
plt.scatter(train3['Year Built'], train3['SalePrice'])
plt.title('Year Built vs Sale Price', size = 20)
plt.xlabel('Year Built', size=15)
plt.ylabel('Sale Price ($)', size=15);

In [ ]:
#Linear coeffs with the ones removed, as designated by the lasso regression
pd.DataFrame({'column':X.columns, 'coef':linreg.coef_})

In [ ]:
#Scatter plot of Kitched Above Grade vs Sale Price
plt.figure(figsize=(10,10))
plt.scatter(train3['Kitchen AbvGr'], train3['SalePrice'])
plt.title('Kitchens Above Grade vs Sale Price', size = 20)
plt.xlabel('Number of Kitchens Above Grade', size=15)
plt.ylabel('Sale Price ($)', size=15)

## Summary
In this notebook, I finally realized the outliers and remove them. I will do this in my first notebook during the next project and the rest of my data science career. I'm glad I learned this lesson now.

I also used this notebook to make my visuals for my presentation, since it was my best model.